In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
bookings = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
comments = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
likes = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
bookings.head()
comments.head()
likes.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,c58d422b-c612-4088-8645-6b65e8de2992,f36541da-a645-40ea-8d0c-dd11452b36ef,545c1990-0afa-4916-9699-bd6201976dfe,7f7acffe-72cd-4342-9a76-b9121d1081fb
1,f4ba7042-6fae-4a7a-ab43-cd20bbcb0979,f3159360-6da4-42f4-99d7-34b078d194da,14257c8a-014a-4a69-a3ea-4bcc57a0396f,340e2498-0efb-4716-b1d3-448c40657b9c
2,5e093549-092a-4341-83a0-6a2164cc84f3,6b3f6145-572e-4494-87f5-5e5891635eff,b173f9c2-3825-4c87-a724-99c2882030e6,b8fb93c9-08cd-4403-817a-2b1650c3b936
3,ed762490-2c8a-4126-aae4-65927fd6c998,0f4b8b49-4acf-42a3-b76c-dd2452c52434,42409373-2ba7-4322-a0b5-1aa437afec57,6d639090-faad-4f44-9662-d3624a16d07f
4,63211825-4aa2-498f-b5df-a853ee0df8a8,0c8e254b-96bd-45a0-bfff-232a3ed50ba6,5a0e9844-ecb1-4ccf-8b50-a6bc118872f5,badb04e3-7c28-4f00-9784-0968fe10f12f


In [7]:
bookings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
comments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245397 entries, 0 to 245396
Data columns (total 4 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   user_id                       245397 non-null  object
 1   commented_event_id            245397 non-null  object
 2   commented_event_organizer_id  245397 non-null  object
 3   comment_id                    245397 non-null  object
dtypes: object(4)
memory usage: 7.5+ MB


In [9]:
likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145397 entries, 0 to 145396
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   user_id                   145397 non-null  object
 1   liked_event_id            145397 non-null  object
 2   liked_event_organizer_id  145397 non-null  object
 3   like_id                   145397 non-null  object
dtypes: object(4)
memory usage: 4.4+ MB


In [10]:
# Standardize column names across all datasets
bookings = bookings.rename(columns={
    'booked_event_id': 'event_id',
    'booked_event_organizer_id': 'organizer_id',
    'booking_id': 'id'
})

comments = comments.rename(columns={
    'commented_event_id': 'event_id',
    'commented_event_organizer_id': 'organizer_id',
    'comment_id': 'id'
})

likes = likes.rename(columns={
    'liked_event_id': 'event_id',
    'liked_event_organizer_id': 'organizer_id',
    'like_id': 'id'
})

# Create source-specific DataFrames with indicator columns
bookings_df = bookings[['user_id', 'event_id', 'organizer_id']].copy()
bookings_df['is_booked'] = 1
bookings_df['num_bookings'] = 1

comments_df = comments[['user_id', 'event_id', 'organizer_id']].copy()
comments_df['is_commented'] = 1
comments_df['num_comments'] = 1

likes_df = likes[['user_id', 'event_id', 'organizer_id']].copy()
likes_df['is_liked'] = 1
likes_df['num_likes'] = 1

# Get all unique user-event pairs
unique_pairs = pd.concat([
    bookings[['user_id', 'event_id']],
    comments[['user_id', 'event_id']],
    likes[['user_id', 'event_id']]
]).drop_duplicates()

# Randomly select 100,000 unique pairs if we have more than that
if len(unique_pairs) > 200000:
    unique_pairs = unique_pairs.sample(n=200000, random_state=42)

# Get consistent organizer_id mapping (most frequent for each event)
organizer_mapping = pd.concat([
    bookings[['event_id', 'organizer_id']],
    comments[['event_id', 'organizer_id']],
    likes[['event_id', 'organizer_id']]
]).groupby('event_id')['organizer_id'].agg(lambda x: x.mode()[0]).to_dict()

# Merge all interaction types
result = unique_pairs.copy()

# Merge booking data
result = result.merge(
    bookings_df.groupby(['user_id', 'event_id']).agg({
        'is_booked': 'max',
        'num_bookings': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge comment data
result = result.merge(
    comments_df.groupby(['user_id', 'event_id']).agg({
        'is_commented': 'max',
        'num_comments': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge like data
result = result.merge(
    likes_df.groupby(['user_id', 'event_id']).agg({
        'is_liked': 'max',
        'num_likes': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Fill NA values with 0 for indicator and count columns
for col in ['is_booked', 'num_bookings', 'is_commented', 'num_comments', 'is_liked', 'num_likes']:
    result[col] = result[col].fillna(0).astype(int)

# Consolidate organizer_id (take first non-NA value)
result['organizer_id'] = (result['organizer_id_x']
                         .fillna(result['organizer_id_y'])
                         .fillna(result['organizer_id']))

# If still missing, use our mapping
result['organizer_id'] = result['organizer_id'].fillna(result['event_id'].map(organizer_mapping))

# Select final columns
final_data = result[[
    'user_id', 
    'event_id', 
    'is_booked',
    'num_bookings',
    'is_liked',
    'num_likes',
    'is_commented', 
    'num_comments', 
    'organizer_id'
]]

# Ensure we have exactly 100,000 records (in case some merges added duplicates)
final_data = final_data.drop_duplicates().sample(n=min(200000, len(final_data)), random_state=42)

# Save the merged data
final_data.to_csv("artifacts/data_ingestion/generated_data/merged_interactions.csv", index=False)

print("Merged data saved successfully!")
print(f"Final dataset shape: {final_data.shape}")
print(final_data.head())

Merged data saved successfully!
Final dataset shape: (200000, 9)
                                     user_id  \
119737  f8b4b424-e696-46bb-924f-baec27b9fc0b   
72272   c07b8075-609c-4a1c-8d38-cb2863f38bda   
158154  96da84b0-4dc9-481e-be8c-7c65d12c317b   
65426   706a00bf-d542-4e7a-96bf-a158b7dc004d   
30074   069c6f09-ffbb-41c7-b813-06d7b9f11b5d   

                                    event_id  is_booked  num_bookings  \
119737  ec8ff257-3f56-43fc-a8e2-7e9730cb4f1f          0             0   
72272   01a5a968-4ae3-42f5-9ce4-edd11b7595d4          0             0   
158154  a48a16a6-b307-4c70-9f3e-aa618d6da9fb          1             4   
65426   f937f03c-fe81-4359-a7a9-79cf0185c5d0          0             0   
30074   8b366abc-c53b-4b85-8881-1ecdb9af2a9a          0             0   

        is_liked  num_likes  is_commented  num_comments  \
119737         0          0             1             1   
72272          0          0             1             3   
158154         0          0   